# Retrain Model to accept only person class

### To extract just pedestrian data from frames

Notebook currently does not work and sticks on first training epoch when using updated mrcnn from https://github.com/leekunhee/Mask_RCNN

When investigated the identification from this model does not work the same as matterports original 

When trying to retrain matterports original model, errors are encountered due to changes within versions of Keras and TensorFlow

In [1]:
import os
import sys
import numpy as np
np.random.seed(0)
import matplotlib.pyplot as plt
import cv2
import re

%matplotlib inline 

# Root directory of the project
ROOT_DIR = os.path.abspath("../")
print("Root Directory: ", ROOT_DIR)
sys.path.append(ROOT_DIR)

from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

from scripts import coco
from scripts.coco import Config
from scripts import list_file_info as lfi

from pycocotools.coco import COCO

Root Directory:  /home/sam/Documents/GitHub/honoursProject/PedestrianDetectionSGall


In [2]:
vid_dir = "/home/sam/Desktop/sam_bdd_videos/videos_for_frames/1/video/"
frames_dir = "/home/sam/Desktop/sam_bdd_videos/videos_for_frames/1/frames/"
output_dir = "/home/sam/Desktop/sam_bdd_videos/videos_for_frames/1/output_vid/"
final_imgs_dir = "/home/sam/Desktop/sam_bdd_videos/videos_for_frames/1/output_img/"
json_output_dir = "/home/sam/Desktop/sam_bdd_videos/videos_for_frames/1/output_json/"
COCO_DIR = "/home/sam/Desktop/cocoData/"
COCO_MODEL_PATH = "/home/sam/Documents/GitHub/honoursProject/PedestrianDetectionSGall/weights/mask_rcnn_coco.h5"
MODEL_DIR = "/home/sam/Documents/GitHub/honoursProject/PedestrianDetectionSGall/weights/"

In [3]:
#class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    #GPU_COUNT = 1
    #IMAGES_PER_GPU = 1

# Alt Config to train own model on reduced classes 
class PersonConfig(Config):
    NAME = "person"
    NUM_CLASSES = 1 + 1 # background + sheep

config = PersonConfig()  # Don't forget to use this config while creating your model
config.display()


# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [4]:
ct = COCO("/home/sam/Desktop/cocoData/annotations/instances_train2014.json")
print(ct.getCatIds(['BG']))
print(ct.getCatIds(['person']))

loading annotations into memory...
Done (t=8.17s)
creating index...
index created!
[]
[1]


In [5]:
# Training dataset
dataset_train = coco.CocoDataset()
dataset_train.load_coco(COCO_DIR, "train", class_ids=[1])
dataset_train.prepare()

# Validation dataset
dataset_val = coco.CocoDataset()
dataset_val.load_coco(COCO_DIR, "val", class_ids=[1])
dataset_val.prepare()

loading annotations into memory...
Done (t=8.11s)
creating index...
index created!
loading annotations into memory...
Done (t=4.11s)
creating index...
index created!


In [6]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config, model_dir=MODEL_DIR)
model.load_weights(COCO_MODEL_PATH, by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])

TypeError: Could not build a TypeSpec for KerasTensor(type_spec=TensorSpec(shape=(None, None, 4), dtype=tf.float32, name=None), name='tf.math.truediv/truediv:0', description="created by layer 'tf.math.truediv'") of unsupported type <class 'keras.engine.keras_tensor.KerasTensor'>.

In [ ]:
model.train(dataset_train, dataset_val,
        learning_rate=config.LEARNING_RATE, 
        epochs=100, 
        layers='heads')

In [ ]:
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "weights/mask_rcnn_coco.h5")

# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

class_names = ['BG','person']



In [ ]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

print("Loading weights ", COCO_MODEL_PATH)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

In [ ]:
# Create collection for frame names in frames_dir

frames_list = lfi.get_file_names(frames_dir)

# Sort the frames into accending order

frames_list.sort(key=lambda f: int(re.sub('\D', '', f)))

In [ ]:
test_annos = dict()


for i in range(9):
    
    image_path = os.path.join(frames_dir,frames_list[i]) 
    print(image_path)
    image = cv2.imread(image_path)
    _, ax = plt.subplots(1, figsize=(26, 26))
    
    height, width = image.shape[:2]
    
    results = model.detect([image], verbose=1)
    
    r = results[0]
    
    image_ir = visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                                class_names, r['scores'], ax=ax,
                                title="Predictions")
    
    plt.savefig(os.path.join(output_dir,frames_list[i]), bbox_inches='tight')
    
    test_annos[i] = {frames_list[i] : {'objects': ([class_names[predicted_class_name] for predicted_class_name in r['class_ids']], 
                                                r['scores'].tolist(),  
                                                r['rois'].tolist(),
                                                class_names,
                                                width, height)}}
    if i==9:
        break